<a href="https://colab.research.google.com/github/michelup/TEST_SDA/blob/main/Projekt_2_15052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
pip install ydata-profiling


In [174]:
pip install lazypredict


In [175]:
import kagglehub
import os
import pandas as pd

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from lazypredict.Supervised import LazyClassifier

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn import tree
import graphviz
import seaborn as sns

In [176]:
# Download latest version
slozka = kagglehub.dataset_download("ahsan81/hotel-reservations-classification-dataset")

In [177]:
soubor = os.listdir(slozka)[0]

In [178]:
data = pd.read_csv(slozka + os.sep + soubor, index_col="Booking_ID")

In [179]:
data

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled


In [180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

In [181]:
data.describe( )

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00
mean,1.84,0.11,0.81,2.20,0.03,85.23,2017.82,7.42,15.60,0.03,0.02,0.15,103.42,0.62
std,0.52,0.40,0.87,1.41,0.17,85.93,0.38,3.07,8.74,0.16,0.37,1.75,35.09,0.79
min,0.00,0.00,0.00,0.00,0.00,0.00,2017.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,2.00,0.00,0.00,1.00,0.00,17.00,2018.00,5.00,8.00,0.00,0.00,0.00,80.30,0.00
50%,2.00,0.00,1.00,2.00,0.00,57.00,2018.00,8.00,16.00,0.00,0.00,0.00,99.45,0.00
75%,2.00,0.00,2.00,3.00,0.00,126.00,2018.00,10.00,23.00,0.00,0.00,0.00,120.00,1.00
max,4.00,10.00,7.00,17.00,1.00,443.00,2018.00,12.00,31.00,1.00,13.00,58.00,540.00,5.00


In [182]:
#POZN  MARKET SEGMENT ROZDELIT DO 3 a zbytek OTHER

# Define the column you want to modify
column_to_modify = 'market_segment_type'

# Define the list of specific values to group into 'Other'
values_to_group = ['Corporate', 'Aviation', 'Complementary']

# Check the current value counts before grouping (optional but recommended)
print(f"Original value counts for '{column_to_modify}':")
print(data[column_to_modify].value_counts())

# Use the replace method to change the specified values to 'Other'
# Using .loc to ensure proper assignment and avoid
data.loc[:, column_to_modify] = data[column_to_modify].replace(values_to_group, 'Other')

# Check the value counts after grouping to verify the change
print(f"\nValue counts for '{column_to_modify}' after grouping:")
print(data[column_to_modify].value_counts())

# Display the head of the DataFrame to see the changes (optional)
column_to_show = 'market_segment_type'
print(f"\nUnique values in the '{column_to_show}' column:")
print(data[column_to_show].unique())

Original value counts for 'market_segment_type':
market_segment_type
Online           23214
Offline          10528
Corporate         2017
Complementary      391
Aviation           125
Name: count, dtype: int64

Value counts for 'market_segment_type' after grouping:
market_segment_type
Online     23214
Offline    10528
Other       2533
Name: count, dtype: int64

Unique values in the 'market_segment_type' column:
['Offline' 'Online' 'Other']


In [183]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

EDA

In [184]:
EDA_HTML = False

FILTER_KORELACE = False
THRESHOLD = 0.2

DOWNSAMPLING = False
UPSAMPLING = True
SMOTE = False # if true -> smote, else adasyn

In [185]:
df=data.reset_index()

In [186]:
avg_prices = df.groupby('room_type_reserved')['avg_price_per_room'].mean()
print(avg_prices)  # Zobrazíme průměrné ceny pro každou kategorii
df['avg_price_per_room'] = df.apply(
    lambda row: avg_prices[row['room_type_reserved']] if row['avg_price_per_room'] == 0 else row['avg_price_per_room'], axis=1
)

print(df.head())  # Zobrazíme upravený DataFrame


room_type_reserved
Room_Type 1    95.92
Room_Type 2    87.85
Room_Type 3    73.68
Room_Type 4   125.29
Room_Type 5   123.73
Room_Type 6   182.21
Room_Type 7   155.20
Name: avg_price_per_room, dtype: float64
  Booking_ID  no_of_adults  no_of_children  no_of_weekend_nights  \
0   INN00001             2               0                     1   
1   INN00002             2               0                     2   
2   INN00003             1               0                     2   
3   INN00004             2               0                     0   
4   INN00005             2               0                     1   

   no_of_week_nights type_of_meal_plan  required_car_parking_space  \
0                  2       Meal Plan 1                           0   
1                  3      Not Selected                           0   
2                  1       Meal Plan 1                           0   
3                  2       Meal Plan 1                           0   
4                  1      Not Sele

In [187]:
#prevod na
from sklearn.preprocessing import OneHotEncoder

# Inicializace encoderu
encoder = OneHotEncoder(sparse_output=False)

# Transformace dat
encoded_array = encoder.fit_transform(df[['type_of_meal_plan','room_type_reserved','market_segment_type','booking_status']])

# Vytvoření nového DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

#prevod z float na integer
encoded_df = encoded_df.astype(int)


In [188]:
#novy data frame
encoded_df

#neslo nam spojit tabulky, dokud jsme neresetovali index

df_encoded=encoded_df.reset_index(drop=True)
df_encoded

,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,type_of_meal_plan_Not Selected,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other,booking_status_Canceled,booking_status_Not_Canceled
0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
4,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
36271,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
36272,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
36273,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0


In [189]:
df_reset = df.reset_index(drop=True)

df_reset

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


In [190]:
#pripojeni puvodniho datasetu k one-hot datasetu
df=df_reset.join(encoded_df)

#odstraneni Booking_ID sloupce a puvodniho "type of meal"
df.drop(columns=['type_of_meal_plan','Booking_ID','room_type_reserved','market_segment_type','booking_status'], inplace=True)

df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other,booking_status_Canceled,booking_status_Not_Canceled
0,2,0,1,2,0,224,2017,10,2,0,...,0,0,0,0,0,1,0,0,0,1
1,2,0,2,3,0,5,2018,11,6,0,...,0,0,0,0,0,0,1,0,0,1
2,1,0,2,1,0,1,2018,2,28,0,...,0,0,0,0,0,0,1,0,1,0
3,2,0,0,2,0,211,2018,5,20,0,...,0,0,0,0,0,0,1,0,1,0
4,2,0,1,1,0,48,2018,4,11,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,0,1,0,0,0,0,1,0,0,1
36271,2,0,1,3,0,228,2018,10,17,0,...,0,0,0,0,0,0,1,0,1,0
36272,2,0,2,6,0,148,2018,7,1,0,...,0,0,0,0,0,0,1,0,0,1
36273,2,0,0,3,0,63,2018,4,21,0,...,0,0,0,0,0,0,1,0,1,0


In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   required_car_parking_space            36275 non-null  int64  
 5   lead_time                             36275 non-null  int64  
 6   arrival_year                          36275 non-null  int64  
 7   arrival_month                         36275 non-null  int64  
 8   arrival_date                          36275 non-null  int64  
 9   repeated_guest                        36275 non-null  int64  
 10  no_of_previous_cancellations          36275 non-null  int64  
 11  no_of_previous_

In [192]:

# GET - dummies  TOTO MI DELA BLBOSTI
#room_type_reserved_encoded = pd.get_dummies(data["room_type_reserved"])
#df.join(room_type_reserved_encoded).drop(columns="room_type_reserved")

In [195]:

df[['avg_price_per_room']].min()


,0
avg_price_per_room,0.50


In [196]:
pocet_nul = (df['avg_price_per_room'] == 0).sum()
pocet_nul

np.int64(0)